In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import shutil
from scipy.interpolate import RectBivariateSpline
import time
from IPython.display import clear_output
import queue

#To Clear the model cache
# shutil.rmtree(torch.hub.get_dir(), ignore_errors=True)

midas = torch.hub.load('intel-isl/MiDaS','MiDaS_small')
midas.to('cpu')

#Process image
transforms = torch.hub.load('intel-isl/MiDaS','transforms')
transform = transforms.small_transform

prev = None
cap = cv2.VideoCapture(0)
time.sleep(0.5)

movement = queue.Queue(maxsize=3)
movement.put(0)

while cap.isOpened():
    ret, img = cap.read()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Making a prediction
    with torch.no_grad():
        prediction = midas(transform(img).to('cpu'))
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

    output = prediction.cpu().numpy()
    # output_norm = cv2.normalize(output, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    if prev is None:
        prev = output

    diff = output - prev
    diff_norm = cv2.normalize(diff, None, -0.5, 0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    clear_output(wait=True)
    fig, ax = plt.subplots()
    # scatter red dot at center
    ax.scatter(640, 360, c='r', s=10)
    ax.imshow(output)
    plt.axis('off')
    plt.show()

    if movement.full():
        movement.get()

    if diff_norm[360][640] > 0.15:
        movement.put(1)
    elif diff_norm[360][640] < -0.15:
        movement.put(-1)
    else:
        movement.put(0)

    mean = np.mean(movement.queue)
    if mean > 0.35:
        print(' further')
    elif mean < -0.35:
        print(' closer')
    else:
        print(' stop')
    
    print(1000 - output[360][640])

    # Frame rate
    plt.pause(0.15)
    
    prev = output

